In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import ujson

# 1. 处理Wiki数据

In [2]:
origin_wiki_file = './data/wiki.simple.txt'

liness = []
with open(origin_wiki_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [ ]:
lines[0:5]

In [7]:
# 删除空行
new_lines = []
for line in lines:
    if line.strip() != '':
        new_lines.append(line)

In [ ]:
print(len(lines), len(new_lines))

In [9]:
# with open('./data/wiki.simple_new.txt', 'w', encoding='utf-8') as f:
#     f.writelines(new_lines)

In [ ]:
tb = pa.Table.from_arrays([new_lines], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/wiki.simple.parquet', row_group_size=50000, data_page_size=50000, )

# 2. 处理训练数据

In [10]:
my_data = []
max_len = 384

In [11]:
with open('./data/bell_open_source/train_3.5M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        if len(item['conversations']) != 2: continue

        conversation = item['conversations']
        txt = ''
        if conversation[0]['from'] =='human':
            txt = f"{conversation[0]['value']}\n{conversation[1]['value']}"
        else:
            txt = f"{conversation[1]['value']}\n{conversation[0]['value']}"

        if len(txt) >= max_len: continue
        my_data.append(txt)

In [12]:

for file in ['./data/bell_open_source/train_2M_CN.json', './data/bell_open_source/Belle_open_source_1M.json']:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            item = ujson.loads(line)

            if item['input'].strip() != '':
                txt = f"{item['instruction']}\n{item['input']}\n{item['output']}"
            else:
                txt = f"{item['instruction']}\n{item['output']}"
            
            if len(txt) == 0 or len(txt) >= max_len: continue
            my_data.append(
                    txt
            )

In [ ]:
print(len(my_data), my_data[0:5])


In [18]:
tb = pa.Table.from_arrays([my_data], names=['text'])
# compression='GZIP'
pq.write_table(table=tb, where='./data/bell_pretrain_3M.parquet', row_group_size=20480, data_page_size=20480, )

# 3. 处理sft数据

In [6]:
lines = []
with open('./data/sft_0.8M_CN.json', 'r', encoding='utf-8') as f:
    for line in f:
        item = ujson.loads(line)

        txt = f"{item['instruction']}{item['output']}"
        
        if len(txt) == 0 or len(txt) >= 320: continue
        lines.append(
                item
        )

In [ ]:
print(len(lines))

In [8]:
tb = pa.Table.from_pylist(lines)
# compression='GZIP'
pq.write_table(table=tb, where='./data/sft_train_data.parquet', row_group_size=20480, data_page_size=20480, )